# MVP - Análise Variável Resposta

Notebook reúne o conjunto de análises iniciais dos dados históricos de inadimplência. Basicamente, a 
tabela sendo analisada contêm uma série temporal de contas contrato e, uma flag indicando se aquela 
conta contrato pagou a fatura do mês ou não. Inicialmente o input contêm as seguintes informações:

<ul>
    <li><code>'ZCGACCOUN'</code>: Conta contrato.</li>
    <li><code>'ZCGDOCREF'</code>: Documento de referência.</li>
    <li><code>'SITUCAO_FAT'</code>:Flag de inadimplência.</li>
    <li><code>'DT_VENCIMENTO'</code>: Data de vencimento da fatura.</li>
    <li><code>'DT_PAGAMENTO'</code>: Data de pagamento. Se nula, significa que a conta-ciontrato não pagou a fatura (logo se encontra inadimplente).</li>
    <li><code>'VLR_FATURA'</code>: Valor da fatura.</li>
    <li><code>'FATURAS_NAO_PAGAS'</code>: Quantidade de faturas não pagas.</li>
</ul>


Como temos que a coluna <code>'DT_PAGAMENTO'</code> se encontra vazia para clientes inadimplentes, não podemos utilizar ela como uma feature do 
modelo, uma vez que ela representa exatamente a mesma coisa que a nossa flag <code>'SITUCAO_FAT'</code>. Um jeito mais "simples" de entender 
a lógica do porque precisamos remover ela, é pensando no momento em que ela é gerada (que é basicamente no mesmo momento em que são 
classifcados os clientes que <b>FICARAM</b> inadimplentes e os que <b>PAGARAM</b> em dia).

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ZCGACCOUN</th>
      <th>ZCGDOCREF</th>
      <th>SITUCAO_FAT</th>
      <th>DT_VENCIMENTO</th>
      <th>DT_PAGAMENTO</th>
      <th>VLR_FATURA</th>
      <th>FATURAS_NAO_PAGAS</th>
      <th>TOTAL_DOCREF</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>7030136858</td>
      <td>0202009123694998</td>
      <td><font color=green>PAGO</font></td>
      <td>2020-09-18</td>
      <td>2020-09-30</td>
      <td>18.33</td>
      <td>14</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>7017824761</td>
      <td>0202201191749005</td>
      <td><font color=red>NÃO PAGO</font></td>
      <td>2022-02-04</td>
      <td><font color=red>NaN</font></td>
      <td>136.67</td>
      <td>12</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>4002932550</td>
      <td>0201905062044492</td>
      <td><font color=green>PAGO</font></td>
      <td>2019-05-22</td>
      <td>2019-05-23</td>
      <td>530.95</td>
      <td>6</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4002932550</td>
      <td>0202112185581738</td>
      <td><font color=green>PAGO</font></td>
      <td>2021-12-22</td>
      <td>2021-12-23</td>
      <td>1033.43</td>
      <td>6</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>7015352757</td>
      <td>0201912090657141</td>
      <td><font color=green>PAGO</font></td>
      <td>2020-01-07</td>
      <td>2020-02-17</td>
      <td>50.54</td>
      <td>1</td>
      <td>1</td>
    </tr>
  </tbody>
</table>

In [4]:
from __future__ import annotations

import logging
from pathlib import Path
from typing import Any

import pandas as pd
import sys
sys.path.append("/Users/erikingwersen/Desktop/neo_analytics/src")


from sklearn.model_selection import train_test_split
from preprocessing.encoders.num_encoders import SelectNumerical, DistTransformer
from preprocessing.encoders.cat_encoders import SelectCategorical, LabelEncoder
from preprocessing.pipeline import Pipeline

import autokeras as ak
import tensorflow as tf

In [5]:
import mitosheet
import textwrap

from typing import Iterable, Hashable
import numpy as np


In [6]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
basestring = (str, bytes)

In [54]:
df.to_csv('/Users/erikingwersen/Desktop/neo_analytics/data/FLAG_FAT_PGTO.csv', index=False)

<h3>Leitura de Inputs</h3>

O <code>pandas</code> define um conjunto de funções <code>I/O</code> 
que auxiliam no processo de leitura de dados em dierentes formatos. Em geral,
as funções que vem com a biblioteca (<code>pandas</code>) são extremamenete úteis e,
fazem o trabalho de maneira bem eficionte.


In [7]:



def read_csv(filepath: Path, sep: str = ",") -> pd.DataFrame:
    """
    Lê um arquivo CSV e retorna um DataFrame.

    Durante a leitura, caso exista apenas uma coluna no dataframe,
    a função assume que o separador sendo utilizado é diferente do separador
    do arquivo. Nesse caso, a função irá tentar ler o arquivo com outros separadores
    comumente utilizados.

    Essa função é utilizada internamente pela função `read_file` e não é
    recomendado que ela seja utilizada diretamente.

    Parameters
    ----------
    filepath : Path
        Caminho do arquivo.
    sep : str, optional
        Separador de campos. The default is ','.

    Returns
    -------
    pd.DataFrame
        DataFrame com os dados do arquivo.
    """
    csv_df = pd.read_csv(filepath_or_buffer=filepath, sep=sep)
    if csv_df.shape[1] <= 1:
        other_seps = [";", "|", "\t"]
        for other_sep in other_seps:
            _csv_df = pd.read_csv(filepath_or_buffer=filepath, sep=other_sep)
            if _csv_df.shape[1] > 1:
                logger.info(
                    "Separador diferente foi encontrado durante a leitura do arquivo "
                    f'".csv".\nNovo separador: {other_sep}'
                )
                return _csv_df
    return csv_df


In [8]:



def check_column(
    df: pd.DataFrame, column_names: Iterable | str, present: bool = True
):
    """
    Checks that :param:`df` parameter contains all necessary columns needed.

    Example
    -------
    .. code-block:: python
        check_column(df, ['ZCGACCOUN', 'SITUCAO_FAT'])

    This will check whether columns `'ZCGACCOUN'` and `'SITUCAO_FAT'` exists and 
    columns within `df`'s columns.

    One can also guarantee that `'a'` and `'b'` are not present by switching
    to `present=False`.

    Parameters
    ----------
    df: pd.DataFrame
        The name of the variable.
    column_names: Iterable | str
        A list of column names we want to check to see if
        present (or absent) in `df`.
    present: bool, optional
        If `True` (default), checks to see if all of `column_names`
        are in `df.columns`. If `False`, checks that none of `column_names` are
        in `df.columns`.
    
    Raises
    ------
    ValueError 
        If data is not the expected type.
    """
    if isinstance(column_names, str) or not isinstance(column_names, Iterable):
        column_names = [column_names]

    for column_name in column_names:
        if present and column_name not in df.columns:  # skipcq: PYL-R1720
            raise ValueError(f'{column_name} not present in dataframe columns!')
        elif not present and column_name in df.columns:
            raise ValueError(f'{column_name} already present in dataframe columns!')


def flag_nulls(
    df: pd.DataFrame,
    column_name: Hashable | None = "null_flag",
    columns: str | Iterable[str] | Hashable | None = None,
) -> pd.DataFrame:
    """
    Flag null values inside a dataframe.

    Function creates a new column to indicate whether you have null values
    in a given row. If the :param:`columns` parameter is not set, looks
    the entire DataFrame :param:`df` is used. Otherwise only the indicated
    columns are considered.

    This method does not mutate the original DataFrame.

    Parameters
    ----------
    df: pd.DataFrame
        Input pandas DataFrame.
    column_name: Hashable | None, optional
        Name for the output column.
    columns: str | Iterable[str] | Hashable | None, optional
        List of columns to search for null values. Parameter also
        accepts a single column name, as valid parameter, in case
        there's only one column to be looked at. If set
        to ``None`` (default), all DataFrame columns are used.

    Returns
    -------
    pd.DataFrame
        A copy of the input DataFrame with a new column indicating
        whether there's at least one empty column. This column that gets
        created will either contain zeros or ones.

    Raises
    ------
    ValueError:
        If `column_name` is already present in the DataFrame.
    ValueError:
        If any column within `columns` is not present in the DataFrame.

    Examples
    --------
    Here is an example of what this function does:

    .. code-block:: python

        >>> df = pd.DataFrame({
        ...     "a": ["w", "x", None, "z"], "b": [5, None, 7, 8],
        ... })
        >>> df.flag_nulls()
              a    b  null_flag
        0     w  5.0          0
        1     x  NaN          1
        2  None  7.0          1
        3     z  8.0          0
        >>> df.flag_nulls(columns="b")
              a    b  null_flag
        0     w  5.0          0
        1     x  NaN          1
        2  None  7.0          0
        3     z  8.0          0

    """
    # Sort out columns input
    if isinstance(columns, str):
        columns = [columns]
    elif columns is None:
        columns = df.columns
    elif not isinstance(columns, Iterable):
        # catches other hashable types
        columns = [columns]

    # Input sanitation checks
    check_column(df, columns)
    check_column(df, [column_name], present=False)

    # This algorithm works best for n_rows >> n_cols. See issue #501
    null_array = np.zeros(len(df))
    for col in columns:
        null_array = np.logical_or(null_array, pd.isna(df[col]))

    df = df.copy()
    df[column_name] = null_array.astype(int)
    return df


In [9]:


def is_number(s: Any) -> bool:
    """
    Returns ``True`` is string is a number.

    Parameters
    ----------
    s : Any
        String to check.

    Returns
    -------
    bool
        Returns ``True`` if string is a number. Otherwise ``False``.
    """
    try:
        float(s)
        return True
    except ValueError:
        return False


def fix_numerical_cols(_df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the raw dataframe, converting decimal separator from "," to ".".

    Parameters
    ----------
    _df : pd.DataFrame
        Raw dataframe.

    Returns
    -------
    pd.DataFrame
        Processed dataframe.
    """
    str_df = _df.select_dtypes(include=["object"])
    for col in str_df.columns:
        series_obj = str_df[col].str.replace(
            '.', '', regex=False
        ).str.replace(',', '.', regex=False)
        if series_obj.apply(is_number).all():
            print(f"Column {col} contains numeric data.")
            try:
                _df[col] = series_obj.astype(float)
            except TypeError:
                logger.error(
                    f'Failed to convert column "{col}" to number.'
                )
    return _df

In [10]:


def flatten_iter(iterable):
    """``flatten_iter()`` yields all the elements from *iterable* while
    collapsing any nested iterables.
    >>> nested = [[1, 2], [[3], [4, 5]]]
    >>> list(flatten_iter(nested))
    [1, 2, 3, 4, 5]
    """
    for item in iterable:
        if isinstance(item, Iterable) and not isinstance(item, basestring):
            for subitem in flatten_iter(item):
                yield subitem
        else:
            yield item


def flatten(iterable):
    """``flatten()`` returns a collapsed list of all the elements from
    *iterable* while collapsing any nested iterables.
    >>> nested = [[1, 2], [[3], [4, 5]]]
    >>> flatten(nested)
    [1, 2, 3, 4, 5]
    """
    return list(flatten_iter(iterable))


<h3>Funções de Sumarização</h3>

In [11]:
def display_datashape(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df, pd.DataFrame):
        raise ValueError(
            f'Expected "df" parameter to be a dataframe, not {type(df)}'
        )
    if df.empty:
        print('Dataframe is empty!')
        return df
    nrows, ncols = df.shape
    print(
        'Dataframe successfully read!\n'
        f'- Number of rows: {nrows:,}\n'
        f'- Number of columns: {ncols:,}\n\n'
        'Dataframe contains the following distributions:\n'
        f'{df.describe()}\n\n'
        'Columns data types:\n\n'
        f'{df.info()}\n'
    )
    return df
    

In [12]:
fpath = Path('/Users/erikingwersen/Downloads/faturas flg pagamento.txt').resolve()
df = read_csv(fpath).pipe(fix_numerical_cols).pipe(display_datashape)

Column VLR_FATURA contains numeric data.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365187 entries, 0 to 365186
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ZCGACCOUN          365187 non-null  int64  
 1   ZCGDOCREF          365187 non-null  object 
 2   SITUCAO_FAT        365187 non-null  object 
 3   DT_VENCIMENTO      365187 non-null  object 
 4   DT_PAGAMENTO       350106 non-null  object 
 5   VLR_FATURA         365187 non-null  float64
 6   FATURAS_NAO_PAGAS  365187 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 19.5+ MB
Dataframe successfully read!
- Number of rows: 365,187
- Number of columns: 7

Dataframe contains the following distributions:
          ZCGACCOUN    VLR_FATURA  FATURAS_NAO_PAGAS
count  3.651870e+05  365187.00000      365187.000000
mean   4.729738e+09     143.29874           0.729771
std    2.640772e+09     966.00595           3.777943
min   

## Preparação das Features

De modo geral, a preparação das features consiste na realização de três macroetapas:

1. Conversão dos tipos de dados.
2. Tratamento de campos vazios. Sendo que em geral, existem duas existratégias comumente empregadas: preenchimento ou remoção.
3. Transformação e combinação de features.
4. Normalização dos dados.


### Tipos de Dados

Existem diversos tipos de dados, que podem ser utilizados para representar valores em uma tabela. No entando, podemos dividir 
as variáveis utilizadas para dados tabulares em dois grandes grupos: variáveis categoricas e numéricas. Como o próprio nome 
sugere, valores numéricos são representados por número e, fornecem ideia de "grandeza". Por outro lado, registros categóricos, 
dão uma ideia mais voltada 

Convertendo os valores das colunas <code>"DT_PAGAMENTO"</code> e <code>"DT_VENCIMENTO"</code> para datas. Configurando o parâmetro <code>errros='coerce'</code>, automaticamente converte para <code>pd.NaT</code> valores que a função <code>pd.to_datetime</code> não é capaz de inferir valores de datas.

Com base nas variáveis de data, as seguintes colunas estão sendo criadas:

'MES_VENCIMENTO' Conjunto mês/ano ("YYYYMM") referente à data de vencimento da fatura. Objectivo auxliar na agregação de valores


'DIAS_ATRASO' representa o número de dias com a última fatura em atraso foi paga (caso já tenha sido). Valores negativos, indicam períodos de inadimplência e,
positivos, faturas adiantadas.


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>VLR_FATURA</th>
      <th>FATURAS_NAO_PAGAS</th>
      <th>DIAS_ATRASO</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>count</th>
      <td>365187.00000</td>
      <td>365187.000000</td>
      <td>350106</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>143.29874</td>
      <td>0.729771</td>
      <td>-17 days +23:36:52.095765283</td>
    </tr>
    <tr>
      <th>std</th>
      <td>966.00595</td>
      <td>3.777943</td>
      <td>39 days 19:15:37.328001816</td>
    </tr>
    <tr>
      <th>min</th>
      <td>0.000000</td>
      <td>-1124 days +00:00:00</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>40.43000</td>
      <td>0.000000</td>
      <td>-23 days +00:00:00</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>86.84000</td>
      <td>0.000000</td>
      <td>-7 days +00:00:00</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>161.04000</td>
      <td>0.000000</td>
      <td>0 days 00:00:00</td>
    </tr>
    <tr>
      <th>max</th>
      <td>105975.32000</td>
      <td>90.000000</td>
      <td>61 days 00:00:00</td>
    </tr>
  </tbody>
</table>

In [13]:
df['DT_VENCIMENTO'] = pd.to_datetime(df['DT_VENCIMENTO'], errors='coerce')
df['DT_PAGAMENTO'] = pd.to_datetime(df['DT_PAGAMENTO'], errors='coerce')
df['MES_VENCIMENTO'] = df['DT_VENCIMENTO'].dt.strftime('%Y%m')
df['DIAS_ATRASO'] = df['DT_VENCIMENTO'] - df['DT_PAGAMENTO']
df['SEMANAS_ATRASO'] = df['DIAS_ATRASO'].apply(lambda x: round(x.components[0]/7, 2) if pd.notnull(x) else 9999)

In [14]:
df['MES_VENCIMENTO'] = df['DT_VENCIMENTO'].dt.strftime('%Y%m')

In [15]:
df['SITUCAO_FAT'] = df['SITUCAO_FAT'].apply(lambda x: True if x == 'PAGO' else False)
df

ZCGACCOUN         ZCGDOCREF  SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
0       7030136858  0202009123694998         True    2020-09-18   2020-09-30   
1       7017824761  0202201191749005        False    2022-02-04          NaT   
2       4002932550  0201905062044492         True    2019-05-22   2019-05-23   
3       4002932550  0202112185581738         True    2021-12-22   2021-12-23   
4       7015352757  0201912090657141         True    2020-01-07   2020-02-17   
...            ...               ...          ...           ...          ...   
365182  4010531298  0202103146461868         True    2021-03-15   2021-03-10   
365183  2680764010  0202104151358420         True    2021-04-20   2021-05-06   
365184  1022969028  0202011131254267         True    2020-11-16   2020-12-04   
365185  1294041016  0202010127407152         True    2020-11-17   2020-11-11   
365186  2006046013  0202009123542409         True    2020-09-18   2020-09-25   

        VLR_FATURA  FATURAS_NAO_PAGAS MES_VENCIMENTO DIAS_ATRASO  \
0            18.33                 14         202009    -12 days   
1           136.67                 12         202202         NaT   
2           530.95                  6         201905     -1 days   
3          1033.43                  6         202112     -1 days   
4            50.54                  1         202001    -41 days   
...            ...                ...            ...         ...   
365182       79.81                  0         202103      5 days   
365183       84.40                  0         202104    -16 days   
365184       80.94                  0         202011    -18 days   
365185       75.36                  0         202011      6 days   
365186       78.88                  0         202009     -7 days   

        SEMANAS_ATRASO  
0                -1.71  
1              9999.00  
2                -0.14  
3                -0.14  
4                -5.86  
...                ...  
365182            0.71  
365183           -2.29  
365184           -2.57  
365185            0.86  
365186           -1.00  

[365187 rows x 10 columns]

In [16]:
is_unique = True
cols_to_check_uniqueness = ['ZCGACCOUN', 'ZCGDOCREF', 'DT_VENCIMENTO']
df_no_dups = df[cols_to_check_uniqueness].drop_duplicates()
rows_diff = df.shape[0] - df_no_dups.shape[0]
if rows_diff > 0:
    is_unique = False
    possible_dups = df[~df.index.isin(df_no_dups.index)]
    if not possible_dups.shape[0] == rows_diff:
        raise ValueError(
            'Tem alguma conta errada aí..\n'
            f'Um total de {rows_diff:,} foram removidas '
            'durante a verificação. Mas a operação que gera o '
            'dataframe "possible_dups" achou um total de: '
            f'{possible_dups.shape[0]:,} linhas'
        )
            
    print(
        '\nÉ provável que alguns documentos de referência ("ZCGDOCREF"), '
        'de algumas contas contrato ("ZCGACCOUN") estejam duplicadas.\n'
        'Motivo: Algumas combinações de ' + ', '.join(
            f'"{col}"' for col in cols_to_check_uniqueness
        ).rstrip(', ') + ' possuem mais de um registro.\n\n'
        f'Total de linhas possívelmente duplicadas: {rows_diff:,} '
        f'({rows_diff/df.shape[0]:.2%})\n\nPor exemplo:\n\n'
    )
    print(possible_dups.head(5))

if not is_unique:
    print('\n\n' + '-' * 30 + '\n')
    print('\n\nAnalisando um desses registros possívelmente duplicados:\n\n')
    example = possible_dups.reset_index().loc[0, cols_to_check_uniqueness]
    df_filter = df

    for col in cols_to_check_uniqueness:
        df_filter = df_filter[df_filter[col] == example[col]]
    print(df_filter)


É provável que alguns documentos de referência ("ZCGDOCREF"), de algumas contas contrato ("ZCGACCOUN") estejam duplicadas.
Motivo: Algumas combinações de "ZCGACCOUN", "ZCGDOCREF", "DT_VENCIMENTO" possuem mais de um registro.

Total de linhas possívelmente duplicadas: 2,568 (0.70%)

Por exemplo:


        ZCGACCOUN         ZCGDOCREF  SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
6069   7030465894  0202007114689781        False    2020-07-10          NaT   
8265   2544429011  0202201191754533         True    2022-02-01   2022-01-21   
8454   7008347322  0202201191897009         True    2022-01-31   2022-01-24   
8482   7035129590  0202201189764955         True    2022-01-21   2022-01-11   
11035  7026772855  0201902050091060        False    2019-02-18          NaT   

       VLR_FATURA  FATURAS_NAO_PAGAS MES_VENCIMENTO DIAS_ATRASO  \
6069        17.14                  4         202007         NaT   
8265        21.50                  1         202202     11 days   
8454         1.35        

In [17]:
df_filter

ZCGACCOUN         ZCGDOCREF  SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
6068  7030465894  0202007114689781         True    2020-07-10   2020-07-03   
6069  7030465894  0202007114689781        False    2020-07-10          NaT   

      VLR_FATURA  FATURAS_NAO_PAGAS MES_VENCIMENTO DIAS_ATRASO  SEMANAS_ATRASO  
6068        0.57                  4         202007      7 days             1.0  
6069       17.14                  4         202007         NaT          9999.0

No exemplo acima, parece que podem haver mais de uma linha para o mesmo <code>"ZCGDOCREF"</code>, 
caso o cliente inadimplente quite a dívida em alguma data próxima da do vencimento da fatura.

<br>


<div class="alert alert-block alert-info">
    <b>Info:</b> Talvez possamos filtrar esses casos, aplicando um filtro que busca a 
    data de pagamento (<code>"DT_PAGAMENTO"</code>) mais recente de cada 
    <code>"ZCGDOCREF"/"ZCGACCOUN"</code>.
</div>


<br>

-----

<h2>Exemplo</h2>

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th><b>ZCGACCOUN</b></th>
      <th><b>ZCGDOCREF</b></th>
      <th><b>SITUCAO_FAT</b></th>
      <th><b>DT_VENCIMENTO</b></th>
      <th><b>DT_PAGAMENTO</b></th>
      <th><b>VLR_FATURA</b></th>
      <th><b>FATURAS_NAO_PAGAS</b></th>
      <th><b>MES_VENCIMENTO</b></th>
      <th><b>DIAS_ATRASO</b></th>
      <th><b>SEMANAS_ATRASO</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>6068</th>
      <td><font color=red>7030465894</font></td>
      <td><font color=red>0202007114689781</font></td>
      <td>True</td>
      <td>2020-07-10</td>
      <td>2020-07-03</td>
      <td>0.57</td>
      <td>4</td>
      <td>202007</td>
      <td>7 days</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>6069</th>
      <td><font color=red>7030465894</font></td>
      <td><font color=red>0202007114689781</font></td>
      <td>False</td>
      <td>2020-07-10</td>
      <td>NaT</td>
      <td>17.14</td>
      <td>4</td>
      <td>202007</td>
      <td>NaT</td>
      <td>9999.0</td>
    </tr>
  </tbody>
</table>


<br>

----

Para aplicar o filtro mencionado, podemos simplesmente aplicar um ordenamento antes de realizar 
a remoção das duplicatas. A expressão ficaria assim:


```python

                     1                                                 2

df.sort_values(cols_to_check_uniqueness, ascending=False).drop_duplicates(subset=cols_to_check_uniqueness, keep='first')
   ^^^^^^^^^^^                                            ^^^^^^^^^^^                                      ^^^^^^^^^^^
   | As linhas vão ser ordenadas com base na              | Passando o subset de colunas
   | colunas que estamos utilizando como chave            | que devem ser utilizadas para
   |==========================================            | buscar por linhas duplicadas.
                                                          |===============================                

```

Testando o filtro com o exemplo que utilizamos acima:

In [19]:
df.sort_values(
    cols_to_check_uniqueness,
    ascending=False
).drop_duplicates(
    subset=cols_to_check_uniqueness
).query("ZCGACCOUN == 7030465894 and ZCGDOCREF == '0202007114689781'")


ZCGACCOUN         ZCGDOCREF  SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
6068  7030465894  0202007114689781         True    2020-07-10   2020-07-03   

      VLR_FATURA  FATURAS_NAO_PAGAS MES_VENCIMENTO DIAS_ATRASO  SEMANAS_ATRASO  
6068        0.57                  4         202007      7 days             1.0

Encapsulando o novo filtro:



In [20]:
def make_df_keys_unique(
    df: pd.DataFrame, 
    keys: Hashable | Iterable[Hashable] | None = None,
    sort_keys: bool = True,
    verbose: bool = True,
) -> pd.DataFrame:
    if keys is None:
        keys = list(df.columns)
    if sort_keys:
        df = df.sort_values(keys, ascending=False)
    new_df = df.drop_duplicates(subset=keys, keep='first')
    rows_removed = df.shape[0] - new_df.shape[0]
    print(
        f'\nRemovidas {rows_removed:,} linhas dos inputs '
        'devido à chaves duplicadas.'
    )
    return new_df

In [21]:
df_res = make_df_keys_unique(df, cols_to_check_uniqueness)
df_res


Removidas 2,568 linhas dos inputs devido à chaves duplicadas.


ZCGACCOUN         ZCGDOCREF  SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
71410   7035545764  0202201189216018        False    2022-01-14          NaT   
190098  7035545764  0202112184825904         True    2021-12-16   2022-01-21   
173628  7035545764  0202111180292333         True    2021-11-16   2021-12-23   
7768    7035545764  0202110176081206         True    2021-10-18   2021-10-12   
169727  7035545764  0202109171879688         True    2021-09-16   2021-11-10   
...            ...               ...          ...           ...          ...   
58257       472026  0201905060348415         True    2019-05-10   2019-05-17   
202779      472026  0201904056597110         True    2019-04-10   2019-04-05   
24498       472026  0201903052730843         True    2019-03-14   2019-03-12   
59325       472026  0201902048916648         True    2019-02-11   2019-02-11   
282415      472026  0201901045139298         True    2019-01-11   2019-01-14   

        VLR_FATURA  FATURAS_NAO_PAGAS MES_VENCIMENTO DIAS_ATRASO  \
71410       129.93                  1         202201         NaT   
190098      133.55                  1         202112    -36 days   
173628      122.41                  0         202111    -37 days   
7768        117.96                  1         202110      6 days   
169727      100.25                  1         202109    -55 days   
...            ...                ...            ...         ...   
58257       131.00                  0         201905     -7 days   
202779      145.78                  0         201904      5 days   
24498       141.72                  0         201903      2 days   
59325       136.85                  0         201902      0 days   
282415      118.29                  0         201901     -3 days   

        SEMANAS_ATRASO  
71410          9999.00  
190098           -5.14  
173628           -5.29  
7768              0.86  
169727           -7.86  
...                ...  
58257            -1.00  
202779            0.71  
24498             0.29  
59325             0.00  
282415           -0.43  

[362619 rows x 10 columns]

<h3>Removendo Colunas</h3>

Agora que formatamos os dados do input, criamos outras colunas que podem vir
impactar positivamente a modelagem, só falta removermos as colunas que 
não devem, entrar para a otimização e depois normalizar os dados "pós-processados".

<div class="alert alert-block alert-info">
    <b>Info:</b> Eu estou fazendo isso somente nessa etapa e, não no início 
    da execução, porque acredtio que alguma(s) dessas colunas que iremos remover,
    são utilizadas para o cálculo das demais.
</div>

In [22]:
cols_to_drop = ['ZCGDOCREF', 'DT_PAGAMENTO', 'DIAS_ATRASO', 'SEMANAS_ATRASO']

df_to_model = df_res.drop(columns=cols_to_drop, errors='ignore')

# Acrescentando "0"s como prefixo dos códigos de conta contrato e convertendo 
# a coluna para texto, já que os valores da coluna se referem à códigos de contrato,
# que não possuem um significado ordinal.
df_to_model['ZCGACCOUN'] = df_to_model['ZCGACCOUN'].astype(str).str.zfill(12)

<h2>Normalizando os Dados</h2>

Durante a criação de um modelo preditivo, 
é muito comum que diversas informações, tanto sobre a fatura, quanto 
sobre o cliente acabem servindo como parâmetros de entrada pro modelo.
Por mais relevantes que sejam essas informações de entrada, nós normalmente 
acabamos nos deparando com valores (para um mesmo registro, ou linhas), que possuem 
as mais variadas unidades de media. Em geral, é recomendado que para esses casos, primeiramente seja 
feito uma normalização desses dados. O exemplo hipotético abaixo, mostra um jeito intuitivo (pelo menos pra mim 😁)
de entender a razão (uma das na verdade), de se aplicar essa etapa:


Imagina que você esteja criando um modelo de clusterização de clientes, dentre as variáveis que 
estão sendo utilizadas, você tem uma delas representando a renda anual do cliente (em $\$$ ou $R\$$) e outra 
com a idade do cliente (em anos):

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th><b>CLIENTE</b></th>
      <th><b>COD_CLI</b></th>
      <th><b>IDADE</b></th>
      <th><b>RENDA</b></th>
      <th><b>...</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>ERIK INGWERSEN</td>
      <td>123456</td>
      <td>24</td>
      <td>350.000,00</td>
      <td>...</td>
    </tr>
    <tr>
    <th>2</th>
      <td>MARINA TORQUATO</td>
      <td>78910</td>
      <td>25</td>
      <td>500.000,00</td>
      <td>...</td>
     </tr>
      <tr>
     <th></th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
     </tr>
      <tr>
    <th>5000</th>
      <td>JULIANA ARAUJO</td>
      <td>654321</td>
      <td>26</td>
      <td>700.000,00</td>
      <td>...</td>
    </tr>
  </tbody>
</table>


Supondo que você escolhesse criar o seu modelo utlizando um método como o KMeans. 
Caso os dados não sejam normalizados antes de serem introduzidos para o treino do modelo,
existe uma grande probabilidade do modelo acabar criando os clusters, quase que exclusivamente em cima 
da renda dos clientes. Isso aconteceria, porque alterações nos centróides dos cluster em relação à renda,
acabam al

In [23]:
from preprocessing.selector.elimination import DuplicatedFeatureEliminator, ConstantFeatureEliminator

In [25]:
num_cols_encoder = Pipeline(
    [
        DuplicatedFeatureEliminator(),
        ConstantFeatureEliminator(),
        SelectNumerical(),
        DistTransformer(transform='min-max'),
    ]
)
        
num_df = num_cols_encoder.fit_transform(df_to_model).applymap(lambda x: round(x, 4))

In [26]:
from preprocessing.encoders.cat_encoders import *

In [27]:
cat_cols_encoder = Pipeline(
    [
        DuplicatedFeatureEliminator(),
        ConstantFeatureEliminator(),
        SelectCategorical(), 
        LabelEncoder()
    ]
)
        
cat_df = cat_cols_encoder.fit_transform(df_to_model)
cat_df


ZCGACCOUN DT_VENCIMENTO MES_VENCIMENTO  ZCGACCOUN_le  \
71410   007035545764    2022-01-14         202201             0   
190098  007035545764    2021-12-16         202112             0   
173628  007035545764    2021-11-16         202111             0   
7768    007035545764    2021-10-18         202110             0   
169727  007035545764    2021-09-16         202109             0   
...              ...           ...            ...           ...   
58257   000000472026    2019-05-10         201905          9999   
202779  000000472026    2019-04-10         201904          9999   
24498   000000472026    2019-03-14         201903          9999   
59325   000000472026    2019-02-11         201902          9999   
282415  000000472026    2019-01-11         201901          9999   

        DT_VENCIMENTO_le  MES_VENCIMENTO_le  
71410                  0                  0  
190098                 1                  1  
173628                 2                  2  
7768                   3                  3  
169727                 4                  4  
...                  ...                ...  
58257                715                 34  
202779               706                 35  
24498                736                 36  
59325                751                 37  
282415               768                 38  

[362619 rows x 6 columns]

In [32]:
num_df['ZCGACCOUN'] = cat_df['ZCGACCOUN']
num_df['DT_VENCIMENTO'] = cat_df['DT_VENCIMENTO']
data = num_df

from importlib import reload
from preprocessing.encoders import cat_encoders
reload(cat_encoders)
from preprocessing.encoders import cat_encoders

In [30]:
new_cat_cols = list(set(cat_df.columns) - set(df_to_model.columns))
old_cat_cols = [
    col for col in df_to_model.columns 
    if any(col in new_col for new_col in new_cat_cols)
]
df_to_model = df_to_model.drop(columns=old_cat_cols, errors='ignore').rename(
    columns={
        new_name: [
            old_name for old_name in old_cat_cols 
            if old_name in new_name
        ][0] for new_name in new_cat_cols
    }
)


In [33]:
data

SITUCAO_FAT  VLR_FATURA  FATURAS_NAO_PAGAS     ZCGACCOUN DT_VENCIMENTO
71410           0.0      0.0437             0.0111  007035545764    2022-01-14
190098          1.0      0.0437             0.0111  007035545764    2021-12-16
173628          1.0      0.0436             0.0000  007035545764    2021-11-16
7768            1.0      0.0436             0.0111  007035545764    2021-10-18
169727          1.0      0.0434             0.0111  007035545764    2021-09-16
...             ...         ...                ...           ...           ...
58257           1.0      0.0437             0.0000  000000472026    2019-05-10
202779          1.0      0.0438             0.0000  000000472026    2019-04-10
24498           1.0      0.0438             0.0000  000000472026    2019-03-14
59325           1.0      0.0437             0.0000  000000472026    2019-02-11
282415          1.0      0.0436             0.0000  000000472026    2019-01-11

[362619 rows x 5 columns]

In [35]:
target = 'SITUCAO_FAT'
nrows = data[data[target] == 0].shape[0]
unique_vals = data[target].unique()
data_prop = pd.concat(
    [data[data[target] == cat].iloc[0:nrows] for cat in unique_vals]
)
data_prop

SITUCAO_FAT  VLR_FATURA  FATURAS_NAO_PAGAS     ZCGACCOUN DT_VENCIMENTO
71410           0.0      0.0437             0.0111  007035545764    2022-01-14
345713          0.0      0.0530             0.0111  007035540193    2022-01-17
163907          0.0      0.0437             0.0111  007035539268    2022-02-04
330901          0.0      0.0437             0.0111  007035539268    2022-01-07
45639           0.0      0.0430             0.0000  007035531712    2022-02-11
...             ...         ...                ...           ...           ...
199003          1.0      0.0427             0.0000  007033138401    2021-01-27
338181          1.0      0.0428             0.0000  007033138401    2020-12-29
287479          1.0      0.0427             0.0222  007033138401    2020-09-30
314341          1.0      0.0427             0.0111  007033138401    2020-08-28
197536          1.0      0.0427             0.0000  007033138401    2020-07-27

[27568 rows x 5 columns]

In [36]:
data_prop.shape[0]# == 2 * nrows

27568

In [37]:
RANDOM_STATE = 2018
VALID_SIZE = 0.20

train_df, test_df = train_test_split(
    data_prop, 
    test_size=VALID_SIZE, 
    random_state=RANDOM_STATE, 
    shuffle=True
)
x_train = train_df.drop(columns=[target])
y_train = train_df[target]

x_test = test_df.drop(columns=[target])
y_test = test_df[target]

In [39]:
x2_train = x_train.drop(columns='ZCGACCOUN')

In [40]:
clf = ak.StructuredDataClassifier(
    overwrite=True, 
    multi_label=False,
    metrics=['accuracy', tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.AUC()],
    num_classes=2,
    max_trials=10
)

2022-02-12 20:14:14.891530: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
# Feed the structured data classifier with training data.
clf.fit(x=x2_train, y=y_train.values, epochs=10, validation_split=0.2)

Trial 10 Complete [00h 00m 13s]
val_accuracy: 0.8708428144454956

Best val_accuracy So Far: 0.8724373579025269
Total elapsed time: 00h 02m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
690/690 [==============================] - 2s 1ms/step - loss: 0.4054 - accuracy: 0.8268 - mean_squared_error: 0.1228 - auc: 0.9055
Epoch 2/10
690/690 [==============================] - 1s 1ms/step - loss: 0.3355 - accuracy: 0.8656 - mean_squared_error: 0.1011 - auc: 0.9275
Epoch 3/10
690/690 [==============================] - 1s 1ms/step - loss: 0.3272 - accuracy: 0.8697 - mean_squared_error: 0.0986 - auc: 0.9306
Epoch 4/10
690/690 [==============================] - 1s 1ms/step - loss: 0.3234 - accuracy: 0.8708 - mean_squared_error: 0.0975 - auc: 0.9321
Epoch 5/10
690/690 [==============================] - 1s 1ms/step - loss: 0.3213 - accuracy: 0.8713 - mean_squared_error: 0.0969 - auc: 0.9328
Epoch 6/10
690/690 [==============================] - 1s 2ms/step - loss: 0.3199 - accuracy: 0.8724 - me

2022-02-12 20:17:16.505404: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [42]:
x2_test = x_test.drop(columns='ZCGACCOUN')
y_pred = clf.predict(x=x2_test)

173/173 [==============================] - 0s 1ms/step


In [43]:
x_test['pred'] = flatten(y_pred)
x_test['target'] = y_test

In [6]:
x_test = pd.read_excel('/Users/erikingwersen/Desktop/PREDS_NN_NEP.xlsx')

In [8]:
from importlib import reload
from ml_models.ml_utils import classification
reload(classification)
from ml_models.ml_utils import classification

In [17]:
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append("/Users/erikingwersen/Desktop/neo_analytics/src")

In [9]:
from ml_models.ml_utils.classification import confusion_matrix_visual

import pandas as pd

In [19]:
p = confusion_matrix_visual(x_test['target'], x_test['pred'], class_labels=['INADIMPLENTE', 'NAO INADIMPLENTE'])#, normalize=True)

In [21]:
p.

<AxesSubplot:title={'center':'Confusion matrix'}, xlabel='Actual', ylabel='Model Prediction'>

In [105]:
import mitosheet
x_train['target']=y_train
mitosheet.sheet(x_train, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-6e9126ba-b964-4cd1-a8d2-e33ece646377", "code": {"imports…

In [188]:
piv_df = df.pivot_table(
    columns=['SITUCAO_FAT'],
    index=['ZCGACCOUN', 'MES_VENCIMENTO'],
    values=['FATURAS_NAO_PAGAS', 'VLR_FATURA', 'DIAS_ATRASO', 'ZCGDOCREF'],
    aggfunc={
        'FATURAS_NAO_PAGAS': 'sum',
        'VLR_FATURA': 'sum',
        'DIAS_ATRASO': 'mean',
        'ZCGDOCREF': 'nunique',
    }
)
piv_df

DIAS_ATRASO FATURAS_NAO_PAGAS      VLR_FATURA  \
SITUCAO_FAT                      PAGO          NÃO PAGO PAGO   NÃO PAGO   
ZCGACCOUN  MES_VENCIMENTO                                                 
472026     201901             -3 days               NaN  0.0        NaN   
           201902              0 days               NaN  0.0        NaN   
           201903              2 days               NaN  0.0        NaN   
           201904              5 days               NaN  0.0        NaN   
           201905             -7 days               NaN  0.0        NaN   
...                               ...               ...  ...        ...   
7035545764 202109            -55 days               NaN  1.0        NaN   
           202110              6 days               NaN  1.0        NaN   
           202111            -37 days               NaN  0.0        NaN   
           202112            -36 days               NaN  1.0        NaN   
           202201                 NaT               1.0  NaN     129.93   

                                  ZCGDOCREF       
SITUCAO_FAT                  PAGO  NÃO PAGO PAGO  
ZCGACCOUN  MES_VENCIMENTO                         
472026     201901          118.29       NaN  1.0  
           201902          136.85       NaN  1.0  
           201903          141.72       NaN  1.0  
           201904          145.78       NaN  1.0  
           201905          131.00       NaN  1.0  
...                           ...       ...  ...  
7035545764 202109          100.25       NaN  1.0  
           202110          117.96       NaN  1.0  
           202111          122.41       NaN  1.0  
           202112          133.55       NaN  1.0  
           202201             NaN       1.0  NaN  

[347889 rows x 7 columns]

In [193]:
piv_df = df.pivot_table(
    columns=['SITUCAO_FAT'],
    index=['MES_VENCIMENTO'],
    values=[
        'FATURAS_NAO_PAGAS',
        'VLR_FATURA',
        # 'DIAS_ATRASO',
        'ZCGDOCREF', 
        'ZCGACCOUN'],
    aggfunc={
        'FATURAS_NAO_PAGAS': 'sum',
        'VLR_FATURA': 'mean',
        # 'DIAS_ATRASO': 'mean',
        'ZCGDOCREF': 'nunique',
        'ZCGACCOUN': 'nunique',
    },
    fill_value=0
)
piv_df.sort_index(ascending=False)

FATURAS_NAO_PAGAS         VLR_FATURA             ZCGACCOUN  \
SITUCAO_FAT             NÃO PAGO  PAGO     NÃO PAGO        PAGO  NÃO PAGO   
MES_VENCIMENTO                                                              
202204                         3     0  1516.320000    0.000000         1   
202203                       339    28   344.178441   58.352963       295   
202202                      4236   802   193.784883   72.873772      3387   
202201                      7257  3354   163.912401  158.524778      3382   
202112                      5833  2717   169.489326  181.868765      1021   
202111                      5792  3715   197.123413  168.639077       588   
202110                      4810  3628   205.233085  163.712211       462   
202109                      4929  3882   214.825933  150.709551       430   
202108                      4575  4152   176.621444  147.361676       375   
202107                      4684  4199   104.471568  146.615917       350   
202106                      4163  3733    91.895541  147.498533       292   
202105                      4128  4198    77.468194  143.600455       287   
202104                      3627  3748   186.857893  150.099334       238   
202103                      4096  4023   101.108755  153.903718       253   
202102                      3492  3836    85.280978  143.985397       221   
202101                      3655  3825   110.593378  161.760309       216   
202012                      3164  3730    92.058657  152.845503       197   
202011                      3082  4550    92.615294  143.666894       183   
202010                      3021  5245   146.135690  136.231710       174   
202009                      2898  4935    95.129486  127.496137       171   
202008                      2812  6259   117.944026  126.918236       153   
202007                      2802  7569   297.362654  119.917889       156   
202006                      2608  5320    81.933519  124.796222       157   
202005                      2313  4760    95.798725  143.310497       146   
202004                      2077  4790    95.672857  149.157430       126   
202003                      2714  4254    93.035217  139.517903       144   
202002                      1881  3069   105.018803  138.275085       115   
202001                      2573  3944    95.527162  140.286811       122   
201912                      1803  2775   129.562547  152.369458       105   
201911                      1926  3000   144.514519  148.936611       103   
201910                      1860  3172    92.349904  139.863479       104   
201909                      1808  3231    97.955248  125.073820       100   
201908                      1742  3743    97.580388  124.802516        99   
201907                      1711  5017   108.820198  125.835686        96   
201906                      1445  3232    93.090482  138.574536        83   
201905                      1576  3597    88.291250  144.259041        77   
201904                      1447  3338    84.850896  141.674048        66   
201903                      1359  2871    84.750615  138.158795        65   
201902                      1361  3430    82.220462  126.552043        64   
201901                      1581  3649    90.873377  129.262280        65   

                     ZCGDOCREF         
SITUCAO_FAT     PAGO  NÃO PAGO   PAGO  
MES_VENCIMENTO                         
202204             0         1      0  
202203            27       295     27  
202202           946      3413    957  
202201          6535      3502   6817  
202112          8132      1038   8248  
202111          9006       624   9622  
202110          8632       470   8730  
202109          9136       445   9309  
202108          9275       381   9461  
202107          9315       370   9980  
202106          8866       296   8994  
202105          9474       299  10030  
202104          9049       242   9100  
202103          9544       273  10188  
202102          9082   

In [196]:
float_df = piv_df.select_dtypes(include=['float64'])
piv_df[float_df.columns] = float_df.applymap(lambda x: round(x, 2))
piv_df

FATURAS_NAO_PAGAS       VLR_FATURA         ZCGACCOUN        \
SITUCAO_FAT             NÃO PAGO  PAGO   NÃO PAGO    PAGO  NÃO PAGO  PAGO   
MES_VENCIMENTO                                                              
201901                      1581  3649      90.87  129.26        65  8787   
201902                      1361  3430      82.22  126.55        64  8856   
201903                      1359  2871      84.75  138.16        65  8296   
201904                      1447  3338      84.85  141.67        66  8946   
201905                      1576  3597      88.29  144.26        77  9026   
201906                      1445  3232      93.09  138.57        83  8122   
201907                      1711  5017     108.82  125.84        96  9263   
201908                      1742  3743      97.58  124.80        99  9253   
201909                      1808  3231      97.96  125.07       100  9311   
201910                      1860  3172      92.35  139.86       104  9305   
201911                      1926  3000     144.51  148.94       103  8938   
201912                      1803  2775     129.56  152.37       105  8523   
202001                      2573  3944      95.53  140.29       122  9539   
202002                      1881  3069     105.02  138.28       115  8715   
202003                      2714  4254      93.04  139.52       144  9624   
202004                      2077  4790      95.67  149.16       126  9082   
202005                      2313  4760      95.80  143.31       146  8963   
202006                      2608  5320      81.93  124.80       157  9450   
202007                      2802  7569     297.36  119.92       156  9738   
202008                      2812  6259     117.94  126.92       153  9663   
202009                      2898  4935      95.13  127.50       171  9633   
202010                      3021  5245     146.14  136.23       174  9668   
202011                      3082  4550      92.62  143.67       183  9701   
202012                      3164  3730      92.06  152.85       197  9083   
202101                      3655  3825     110.59  161.76       216  9625   
202102                      3492  3836      85.28  143.99       221  9082   
202103                      4096  4023     101.11  153.90       253  9544   
202104                      3627  3748     186.86  150.10       238  9049   
202105                      4128  4198      77.47  143.60       287  9474   
202106                      4163  3733      91.90  147.50       292  8866   
202107                      4684  4199     104.47  146.62       350  9315   
202108                      4575  4152     176.62  147.36       375  9275   
202109                      4929  3882     214.83  150.71       430  9136   
202110                      4810  3628     205.23  163.71       462  8632   
202111                      5792  3715     197.12  168.64       588  9006   
202112                      5833  2717     169.49  181.87      1021  8132   
202201                      7257  3354     163.91  158.52      3382  6535   
202202                      4236   802     193.78   72.87      3387   946   
202203                       339    28     344.18   58.35       295    27   
202204                         3     0    1516.32    0.00         1     0   

               ZCGDOCREF         
SITUCAO_FAT     NÃO PAGO   PAGO  
MES_VENCIMENTO                   
201901                77  10663  
201902                65   9013  
201903                65   8359  
201904                67   9186  
201905                80   9611  
201906                83   8209  
201907               101  10329  
201908               103   9374  
201909               101   9443  
201910               104   9377  
201911               104   9000  
201912               106   8572  
202001               148  11559  
202002               117   9011  
202003               161  10645  
202004               126   9158  
202005               149   9096  
202006               16

In [200]:
piv_df.columns

MultiIndex([('FATURAS_NAO_PAGAS', 'NÃO PAGO'),
            ('FATURAS_NAO_PAGAS',     'PAGO'),
            (       'VLR_FATURA', 'NÃO PAGO'),
            (       'VLR_FATURA',     'PAGO'),
            (        'ZCGACCOUN', 'NÃO PAGO'),
            (        'ZCGACCOUN',     'PAGO'),
            (        'ZCGDOCREF', 'NÃO PAGO'),
            (        'ZCGDOCREF',     'PAGO')],
           names=[None, 'SITUCAO_FAT'])

In [207]:
acc_count = piv_df.loc[(piv_df.index.astype(int) > 202101) & (piv_df.index.astype(int) <= 202201), [('ZCGACCOUN', 'NÃO PAGO'), ('ZCGACCOUN', 'PAGO')]]

In [219]:
acc_count = piv_df.loc[(piv_df.index.astype(int) > 202101) & (piv_df.index.astype(int) <= 202201), :]

acc_count

FATURAS_NAO_PAGAS       VLR_FATURA         ZCGACCOUN        \
SITUCAO_FAT             NÃO PAGO  PAGO   NÃO PAGO    PAGO  NÃO PAGO  PAGO   
MES_VENCIMENTO                                                              
202102                      3492  3836      85.28  143.99       221  9082   
202103                      4096  4023     101.11  153.90       253  9544   
202104                      3627  3748     186.86  150.10       238  9049   
202105                      4128  4198      77.47  143.60       287  9474   
202106                      4163  3733      91.90  147.50       292  8866   
202107                      4684  4199     104.47  146.62       350  9315   
202108                      4575  4152     176.62  147.36       375  9275   
202109                      4929  3882     214.83  150.71       430  9136   
202110                      4810  3628     205.23  163.71       462  8632   
202111                      5792  3715     197.12  168.64       588  9006   
202112                      5833  2717     169.49  181.87      1021  8132   
202201                      7257  3354     163.91  158.52      3382  6535   

               ZCGDOCREF         
SITUCAO_FAT     NÃO PAGO   PAGO  
MES_VENCIMENTO                   
202102               225   9199  
202103               273  10188  
202104               242   9100  
202105               299  10030  
202106               296   8994  
202107               370   9980  
202108               381   9461  
202109               445   9309  
202110               470   8730  
202111               624   9622  
202112              1038   8248  
202201              3502   6817

In [220]:
import mitosheet
_acc_count = acc_count.reset_index()
mitosheet.sheet(_acc_count, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-c8f17cb1-ef5d-4247-85aa-f8cd776751fb", "code": {"imports…

In [224]:
!pwd

/Users/erikingwersen/Desktop/neo_analytics/src/preprocessing/encoders


In [222]:
_acc_count.to_parquet('./MONTHLY_CONSOLIDATED_VIEW.parquet')

In [203]:
import mitosheet
mitosheet.sheet(piv_df[[('ZCGACCOUN', 'NÃO PAGO'), ('ZCGACCOUN', 'PAGO')]], view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-c598832c-b774-4143-8d0d-c8c99b99e953", "code": {"imports…

In [187]:
import mitosheet
mitosheet.sheet(piv_df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-4016db31-a9f1-4f10-a8f5-822cd8ab97e4", "code": {"imports…

In [184]:
df.columns

Index(['ZCGACCOUN', 'ZCGDOCREF', 'SITUCAO_FAT', 'DT_VENCIMENTO',
       'DT_PAGAMENTO', 'VLR_FATURA', 'FATURAS_NAO_PAGAS', 'MES_VENCIMENTO',
       'DIAS_ATRASO', 'SEMANAS_ATRASO'],
      dtype='object')

In [181]:
import mitosheet
mitosheet.sheet(df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-af6ce39c-ce07-4386-9b11-f99fd309fba1", "code": {"imports…

In [182]:
from mitosheet import *; register_analysis('UUID-af6ce39c-ce07-4386-9b11-f99fd309fba1')

# Pivoted df into df2
unused_columns = df.columns.difference(set(['ZCGACCOUN', 'ZCGDOCREF', 'DT_VENCIMENTO', 'DT_PAGAMENTO']).union(set(['SITUCAO_FAT'])).union(set({'DIAS_ATRASO', 'VLR_FATURA', 'SEMANAS_ATRASO', 'FATURAS_NAO_PAGAS'})))
tmp_df = df.drop(unused_columns, axis=1)
pivot_table = tmp_df.pivot_table(
    index=['ZCGACCOUN', 'ZCGDOCREF', 'DT_VENCIMENTO', 'DT_PAGAMENTO'],
    columns=['SITUCAO_FAT'],
    values=['SEMANAS_ATRASO', 'VLR_FATURA', 'DIAS_ATRASO', 'FATURAS_NAO_PAGAS'],
    aggfunc={'SEMANAS_ATRASO': ['sum'], 'VLR_FATURA': ['sum'], 'DIAS_ATRASO': ['mean'], 'FATURAS_NAO_PAGAS': ['mean']}
)
pivot_table.columns = [flatten_column_header(col) for col in pivot_table.columns.values]
df_pivot = pivot_table.reset_index()

# Renamed FATURAS_NAO_PAGAS mean PAGO to FATURAS_NAO_PAGAS mean PAGOa in df_pivot
df_pivot.rename(columns={'FATURAS_NAO_PAGAS mean PAGO': 'FATURAS_NAO_PAGAS mean PAGOa'}, inplace=True)


In [183]:
df_pivot

ZCGACCOUN         ZCGDOCREF DT_VENCIMENTO DT_PAGAMENTO  \
0           472026  0201901045139298    2019-01-11   2019-01-14   
1           472026  0201902048916648    2019-02-11   2019-02-11   
2           472026  0201903052730843    2019-03-14   2019-03-12   
3           472026  0201904056597110    2019-04-10   2019-04-05   
4           472026  0201905060348415    2019-05-10   2019-05-17   
...            ...               ...           ...          ...   
350101  7035545764  0202108167732913    2021-08-17   2021-10-01   
350102  7035545764  0202109171879688    2021-09-16   2021-11-10   
350103  7035545764  0202110176081206    2021-10-18   2021-10-12   
350104  7035545764  0202111180292333    2021-11-16   2021-12-23   
350105  7035545764  0202112184825904    2021-12-16   2022-01-21   

       DIAS_ATRASO mean PAGO  FATURAS_NAO_PAGAS mean PAGO  \
0                    -3 days                          0.0   
1                     0 days                          0.0   
2                     2 days                          0.0   
3                     5 days                          0.0   
4                    -7 days                          0.0   
...                      ...                          ...   
350101              -45 days                          0.0   
350102              -55 days                          1.0   
350103                6 days                          1.0   
350104              -37 days                          0.0   
350105              -36 days                          1.0   

        SEMANAS_ATRASO sum PAGO  VLR_FATURA sum PAGO  
0                         -0.43               118.29  
1                          0.00               136.85  
2                          0.29               141.72  
3                          0.71               145.78  
4                         -1.00               131.00  
...                         ...                  ...  
350101                    -6.43                95.63  
350102                    -7.86               100.25  
350103                     0.86               117.96  
350104                    -5.29               122.41  
350105                    -5.14               133.55  

[350106 rows x 8 columns]

In [157]:
df.describe()

ZCGACCOUN    VLR_FATURA  FATURAS_NAO_PAGAS  \
count  3.651870e+05  365187.00000      365187.000000   
mean   4.729738e+09     143.29874           0.729771   
std    2.640772e+09     966.00595           3.777943   
min    4.720260e+05   -4702.10000           0.000000   
25%    2.012878e+09      40.43000           0.000000   
50%    7.002810e+09      86.84000           0.000000   
75%    7.020887e+09     161.04000           0.000000   
max    7.035546e+09  105975.32000          90.000000   

                        DIAS_ATRASO  
count                        350106  
mean   -17 days +23:36:52.095765283  
std      39 days 19:15:37.328001816  
min            -1124 days +00:00:00  
25%              -23 days +00:00:00  
50%               -7 days +00:00:00  
75%                 0 days 00:00:00  
max                61 days 00:00:00

In [82]:
num_cols_encoder = Pipeline(
    [
        SelectNumerical(exclude_cols=['ZCGACCOUN']),
        DistTransformer(transform='min-max'),
    ]
)
        
num_df = num_cols_encoder.fit_transform(df2).applymap(lambda x: round(x, 4))

cat_cols_encoder = Pipeline(
    [
        SelectCategorical(), 
        LabelEncoder(
            exclude_cols=[
                'DT_VENCIMENTO', 'DT_PAGAMENTO', 'ZCGDOCREF'
            ]
        )
    ]
)
        
cat_df = cat_cols_encoder.fit_transform(df2)
cat_df

VLR_FATURA  FATURAS_NAO_PAGAS
0           0.0427             0.1556
1           0.0437             0.1333
2           0.0473             0.0667
3           0.0518             0.0667
4           0.0429             0.0111
...            ...                ...
365182      0.0432             0.0000
365183      0.0432             0.0000
365184      0.0432             0.0000
365185      0.0432             0.0000
365186      0.0432             0.0000

[365187 rows x 2 columns]

In [77]:
cat_cols_encoder = Pipeline(
    [SelectCategorical(), LabelEncoder(exclude_cols=['DT_VENCIMENTO', 'DT_PAGAMENTO', 'ZCGDOCREF'])]
)
        
cat_df = cat_cols_encoder.fit_transform(df2)
cat_df

ZCGDOCREF SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
0       0202009123694998        PAGO    2020-09-18   2020-09-30   
1       0202201191749005    NÃO PAGO    2022-02-04          NaN   
2       0201905062044492        PAGO    2019-05-22   2019-05-23   
3       0202112185581738        PAGO    2021-12-22   2021-12-23   
4       0201912090657141        PAGO    2020-01-07   2020-02-17   
...                  ...         ...           ...          ...   
365182  0202103146461868        PAGO    2021-03-15   2021-03-10   
365183  0202104151358420        PAGO    2021-04-20   2021-05-06   
365184  0202011131254267        PAGO    2020-11-16   2020-12-04   
365185  0202010127407152        PAGO    2020-11-17   2020-11-11   
365186  0202009123542409        PAGO    2020-09-18   2020-09-25   

        SITUCAO_FAT_le  
0                    0  
1                    1  
2                    0  
3                    0  
4                    0  
...                ...  
365182               0  
365183               0  
365184               0  
365185               0  
365186               0  

[365187 rows x 5 columns]

In [86]:
new_cat_cols = list(set(cat_df.columns) - set(df2.columns))
old_cat_cols = [
    col for col in df2.columns 
    if any(col in new_col for new_col in new_cat_cols)
]
res_df = res_df.drop(columns=old_cat_cols, errors='ignore').rename(
    columns={
        new_name: [
            old_name for old_name in old_cat_cols 
            if old_name in new_name
        ][0] for new_name in new_cat_cols
    }
)
res_df['DT_VENCIMENTO'] = pd.to_datetime(res_df['DT_VENCIMENTO'], errors='coerce')
res_df['DT_PAGAMENTO'] = pd.to_datetime(res_df['DT_PAGAMENTO'], errors='coerce')

['SITUCAO_FAT']

In [ ]:
res_df = res_df.drop(columns=old_cat_cols, errors='ignore').rename(
    columns={
        new_name: [
            old_name for old_name in old_cat_cols 
            if old_name in new_name
        ][0] for new_name in new_cat_cols
    }
)
res_df['DT_VENCIMENTO'] = pd.to_datetime(res_df['DT_VENCIMENTO'], errors='coerce')
res_df['DT_PAGAMENTO'] = pd.to_datetime(res_df['DT_PAGAMENTO'], errors='coerce')

In [ ]:
df2['MES_VENCIMENTO'] = pd.to_datetime(df2['DT_VENCIMENTO'], errors='coerce').dt.strftime('%Y%m')
df2['TOTAL_DOCREF'] = df2.groupby(['ZCGACCOUN', 'MES_VENCIMENTO'])['ZCGDOCREF'].transform(lambda x: x.nunique())
df2

### TOTAL_DOCREF

Quantidade de faturas `ZCGDOCREF` distintas vinculadas a cada uma das contas contrato da mostra.


In [ ]:
df2['TOTAL_DOCREF'] = df2.groupby(['ZCGACCOUN', 'MES_VENCIMENTO'])['ZCGDOCREF'].transform(lambda x: x.nunique())

In [114]:
df2.query('ZCGACCOUN == 180121013 and MES_VENCIMENTO == 1').sort_values('DT_VENCIMENTO')

ZCGACCOUN         ZCGDOCREF SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
118894  180121013  0201812044377510        PAGO    2019-01-04   2018-12-28   
221282  180121013  0201901048082275        PAGO    2019-01-31   2019-02-01   
293613  180121013  0201912090133639        PAGO    2020-01-06   2020-01-06   
231306  180121013  0202001094115956        PAGO    2020-01-30   2020-01-31   
301748  180121013  0202002098637539        PAGO    2020-01-30   2020-02-27   
211940  180121013  0202012137242693    NÃO PAGO    2021-01-04          NaN   
287256  180121013  0202101141169560    NÃO PAGO    2021-01-29          NaN   
104694  180121013  0202112187449038    NÃO PAGO    2022-01-03          NaN   
137000  180121013  0202201191807985    NÃO PAGO    2022-01-28          NaN   

        VLR_FATURA  FATURAS_NAO_PAGAS  MES_VENCIMENTO  TOTAL_DOCREF  
118894       60.45                  0               1             9  
221282       46.03                  0               1             9  
293613       85.42                  0               1             9  
231306      102.72                  0               1             9  
301748       37.61                  0               1             9  
211940       67.95                  8               1             9  
287256       59.93                  9               1             9  
104694      134.96                 20               1             9  
137000      111.79                 21               1             9

In [108]:
import mitosheet
mitosheet.sheet(df2, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-d4063003-175c-4d85-9f57-d66320de204c", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-d4063003-175c-4d85-9f57-d66320de204c')

# Sorted TOTAL_DOCREF in df2 in descending order
df2 = df2.sort_values(by='TOTAL_DOCREF', ascending=False, na_position='last')


In [101]:
gdf = df2.groupby('ZCGACCOUN', as_index=False).agg({'VLR_FATURA': ['mean', 'std',], 'ZCGDOCREF': 'nunique', 'FATURAS_NAO_PAGAS': 'mean'})
gdf

ZCGACCOUN  VLR_FATURA             ZCGDOCREF FATURAS_NAO_PAGAS
                        mean         std   nunique              mean
0         472026  189.594324   84.167192        37          0.243243
1        1668030  189.832647   48.814761        34          1.000000
2        2607026  263.173846   92.406484        39          0.000000
3        3795012  226.914737   54.071163        38          0.105263
4        3901017   43.100526   14.262617        38          0.184211
...          ...         ...         ...       ...               ...
9995  7035533014  385.110000  166.113811        23          0.173913
9996  7035539268  105.498333   23.521977        24          0.750000
9997  7035540193  675.561739  565.638684        23          0.913043
9998  7035540282   35.182857   16.832813        21          0.142857
9999  7035545764   98.639130   22.745793        23          0.521739

[10000 rows x 5 columns]

In [98]:
import mitosheet
mitosheet.sheet(res_df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-f74f52e5-3203-4660-bd4c-05a56f493472", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-f74f52e5-3203-4660-bd4c-05a56f493472')

# Reordered VLR_FATURA in df1
df1_columns = [col for col in df1.columns if col != 'VLR_FATURA']
df1_columns.insert(4, 'VLR_FATURA')
df1 = df1[df1_columns]

# Reordered VLR_FATURA in df1
df1_columns = [col for col in df1.columns if col != 'VLR_FATURA']
df1_columns.insert(4, 'VLR_FATURA')
df1 = df1[df1_columns]


In [93]:
cols_to_exclude = ['ZCGDOCREF',]
target = 'SITUCAO_FAT'


RANDOM_STATE = 2018
VALID_SIZE = 0.20  # simple validation using train_test_split

train_df, test_df = train_test_split(
    res_df.drop(columns=cols_to_exclude), test_size=VALID_SIZE, random_state=RANDOM_STATE, shuffle=True
)
x_train = train_df.drop(columns=[target])
y_train = train_df[target]

x_test = test_df.drop(columns=[target])
y_test = test_df[target]

clf = ak.StructuredDataClassifier(
    overwrite=True, 
    multi_label=False,
    metrics=['accuracy', tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.AUC()],
    num_classes=2,
    max_trials=10
)

# Feed the structured data classifier with training data.
clf.fit(x=x_train, y=y_train.values, epochs=10, validation_split=0.2)

In [96]:
x_train = train_df.drop(columns=[target])
y_train = train_df[target]

x_test = test_df.drop(columns=[target])
y_test = test_df[target]


28867     0
63388     0
114697    0
219980    1
153869    1
         ..
115605    0
84745     0
342242    0
322150    0
107770    0
Name: SITUCAO_FAT, Length: 292149, dtype: int64

In [97]:
clf = ak.StructuredDataClassifier(
    overwrite=True, 
    multi_label=False,
    metrics=['accuracy', tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.AUC()],
    num_classes=2,
    max_trials=10
)

# Feed the structured data classifier with training data.
clf.fit(x=x_train, y=y_train.values, epochs=10, validation_split=0.2)

Trial 10 Complete [00h 01m 49s]
val_accuracy: 0.9954468607902527

Best val_accuracy So Far: 0.9972270131111145
Total elapsed time: 00h 18m 12s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
9130/9130 [==============================] - 11s 1ms/step - loss: 0.0574 - accuracy: 0.9774 - mean_squared_error: 0.0147 - auc: 0.9890
Epoch 2/10
9130/9130 [==============================] - 11s 1ms/step - loss: 0.0299 - accuracy: 0.9868 - mean_squared_error: 0.0092 - auc: 0.9965
Epoch 3/10
9130/9130 [==============================] - 10s 1ms/step - loss: 0.0252 - accuracy: 0.9891 - mean_squared_error: 0.0077 - auc: 0.9974
Epoch 4/10
9130/9130 [==============================] - 10s 1ms/step - loss: 0.0226 - accuracy: 0.9904 - mean_squared_error: 0.0069 - auc: 0.9979
Epoch 5/10
9130/9130 [==============================] - 13s 1ms/step - loss: 0.0208 - accuracy: 0.9912 - mean_squared_error: 0.0064 - auc: 0.9982
Epoch 6/10
9130/9130 [==============================] - 10s 1ms/step - loss: 0.0198 - acc

2022-02-11 16:17:13.903789: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [84]:
original_df_cols = list(set(df2.columns) - {*cat_df.columns, *num_df.columns})

res_df = pd.concat([cat_df, num_df, df2[original_df_cols]], axis=1)

res_df

ZCGDOCREF SITUCAO_FAT DT_VENCIMENTO DT_PAGAMENTO  \
0       0202009123694998        PAGO    2020-09-18   2020-09-30   
1       0202201191749005    NÃO PAGO    2022-02-04          NaN   
2       0201905062044492        PAGO    2019-05-22   2019-05-23   
3       0202112185581738        PAGO    2021-12-22   2021-12-23   
4       0201912090657141        PAGO    2020-01-07   2020-02-17   
...                  ...         ...           ...          ...   
365182  0202103146461868        PAGO    2021-03-15   2021-03-10   
365183  0202104151358420        PAGO    2021-04-20   2021-05-06   
365184  0202011131254267        PAGO    2020-11-16   2020-12-04   
365185  0202010127407152        PAGO    2020-11-17   2020-11-11   
365186  0202009123542409        PAGO    2020-09-18   2020-09-25   

        SITUCAO_FAT_le  VLR_FATURA  FATURAS_NAO_PAGAS   ZCGACCOUN  
0                    0      0.0427             0.1556  7030136858  
1                    1      0.0437             0.1333  7017824761  
2                    0      0.0473             0.0667  4002932550  
3                    0      0.0518             0.0667  4002932550  
4                    0      0.0429             0.0111  7015352757  
...                ...         ...                ...         ...  
365182               0      0.0432             0.0000  4010531298  
365183               0      0.0432             0.0000  2680764010  
365184               0      0.0432             0.0000  1022969028  
365185               0      0.0432             0.0000  1294041016  
365186               0      0.0432             0.0000  2006046013  

[365187 rows x 8 columns]

In [37]:
import mitosheet
mitosheet.sheet(df2, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-f4f9780d-6231-465b-ad63-8d6b87ec4586", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-f4f9780d-6231-465b-ad63-8d6b87ec4586')

# Changed ZCGACCOUN from int64 to str
df2['ZCGACCOUN'] = df2['ZCGACCOUN'].astype('str')

# Drop duplicates in df2
df2 = df2.drop_duplicates(keep='first')

# Added column new-column-o7ve to df2
df2.insert(3, 'new-column-o7ve', 0)

# Deleted column new-column-o7ve from df2
df2.drop(['new-column-o7ve'], axis=1, inplace=True)


In [15]:
import mitosheet
mitosheet.sheet(df2, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-a2c5144b-5538-4563-8e4d-44532c99812b", "code": {"imports…

In [120]:
df2.attrs['new_categorical_columns'] = new_cat_cols

In [121]:
df2.attrs

{'new_categorical_columns': ['SITUCAO_FAT_le']}

In [122]:
df3 = df2.copy()

In [123]:
df3.attrs

{'new_categorical_columns': ['SITUCAO_FAT_le']}

In [124]:
df4 = df3

In [128]:
df5.attrs

{'new_categorical_columns': ['SITUCAO_FAT_le']}

In [127]:
df5 = df4.groupby('ZCGACCOUN', as_index=False)['VLR_FATURA'].sum()